5단계 까지 도출한 사항을 정리하자면,

Weight Capacity (kg)는 Price를 예측할 수 있는 Key feature 이다. Weight Capacity (kg)는 범주형 성격을 보이는데, 

이 배경에는 동일한 Weight Capacity (kg)에는 동일한 노이즈가 부여되었음이 짐작이 된다. 

이 점은 phase1~5 까지의 여러 실험을 통해 이로 인해 나타나는 결과를 통해 유력함을 확인했습니다. 

단계에서는 이 점을 이용한 Weight Capacity (kg)를  train에 등장하는 Weight Capacity (kg)와 5, 30을 제외한 변수는

Weight Capacity (kg)로 Mean encoding을 입력으로 하는 모델을 이용하고, 

그렇지 않으면 Mean Encoding을 하지 않은 모델을 사용합니다.

In [2]:
import os, sys

import pandas as pd
import polars as pl

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy

import sklearn
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import dproc, sgml, sgnn, sgpp, sgutil, custpp

print(sys.version)
for i in [pd, pl, mpl, sns, np, scipy, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        pass

2025-02-11 08:10:08.593337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739261408.604660   64559 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739261408.608134   64559 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 08:10:08.620137: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]
pandas 2.2.3
polars 1.12.0
matplotlib 3.8.4
seaborn 0.13.2
numpy 1.26.4
scipy 1.12.0
sklearn 1.5.2
xgboost 2.1.2
catboost 1.2.5


In [3]:
from itertools import combinations

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit, train_test_split
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [4]:
sc = sgutil.SGCache('img', 'result')
ss = ShuffleSplit(n_splits = 1, train_size = 0.8, random_state = 123)
kf = KFold(5, random_state = 123, shuffle=True)

files = {
    'train': os.path.join('data', 'train.csv'),
    'train_extra': os.path.join('data', 'training_extra.csv'),
    'test': os.path.join('data', 'test.csv'),
}

t = sc.cache_result(
    'pipeline_2',
    lambda : make_pipeline(
        sgpp.PolarsProcessor(), 
        sgpp.ExprProcessor({
            'Compartments_c' : pl.col('Compartments').cast(pl.String).cast(pl.Categorical)
        }),
        sgpp.PandasCoverter(index_col = 'id'),
        sgpp.ApplyWrapper(
            sgpp.CatArrangerFreq(1, na_value = 'Unknown'),
            ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
        ), 
        custpp.WeightCapacityProcessor()
    ).fit(files['train']),
    rerun = 1
)
df_train = pd.concat(
    [t.transform(files['train']), t.transform(files['train_extra'])], axis = 0
)
df_test = t.transform(files['test'])

target = 'Price'
X_cat = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Compartments_c']
X_num = ['Weight Capacity (kg)']

In [5]:
def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index).clip(15, 150),
    'score_func': lambda df, prds: root_mean_squared_error(df[target].sort_index(), prds.sort_index()),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': target,
}

cb_adapter = sgml.CBAdapter(cb.CatBoostRegressor)
lr_adapter = sgml.SklearnAdapter(LinearRegression)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor)

# CB1

In [6]:
def eval_model(hparams, adapter, case = 0, sp = ss, **args):
    results = list()
    for train_idx, valid_idx in sp.split(df_train, df_train[target]):
        df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
        bidx = df_valid['Weight Capacity (kg)'].isin(df_cv_train['Weight Capacity (kg)'].unique()) & df_valid['Weight Capacity (kg)'].notna() &\
                df_valid['Weight Capacity (kg)'].between(5, 30, inclusive = 'neither')
        if case == 1:
            bidx = ~bidx
        df_valid = df_valid.loc[bidx]
        reg = sgml.train(df_cv_train, hparams, config, adapter, **args)
        prd = make_pipeline(reg[0]['preprocessor'], reg[0]['model']).predict(df_valid[reg[1]])
        results.append(
            (reg, root_mean_squared_error(df_valid[target], prd), pd.Series(prd, index = df_valid.index, name = target))
        )
    return results

In [7]:
results_0 = sc.cache_result(
    'cv_cb1_0',
    lambda : eval_model({
        'model_params' : {'n_estimators': 700, 'max_depth': 7, 'learning_rate': 0.1},
        'X_num': X_num, 'X_cat': X_cat, 'X_tgt': ['wc_i2'], 'tgt': {'cv': 24, 'smooth': 20, 'target_type': 'continuous', 'random_state': 123},
        #'validation_fraction': 0.1
    }, cb_adapter, case = 0, sp = kf, task_type = 'GPU')
)
sgml.cb_learning_result(results_0[0][0][0])['valid_result'].idxmin(), results_0[0][1]

(metric  set  
 RMSE    learn    699
 dtype: int64,
 38.58545493209808)

In [8]:
results_1 = sc.cache_result(
    'cv_cb1_1',
    lambda :eval_model({
        'model_params' : {'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.1},
        'X_num': X_num, 'X_cat': X_cat,
    #    'validation_fraction': 0.1
    }, cb_adapter, case = 1, sp = kf, task_type = 'GPU'), rerun = 0
)
sgml.cb_learning_result(results_1[0][0][0])['valid_result'].idxmin(), results_1[0][1]

(metric  set  
 RMSE    learn    999
 dtype: int64,
 39.21964479050714)

In [9]:
s_cb1 = list()
for a, b in zip(results_0, results_1):
    prd = pd.concat([a[2], b[2]]).sort_index().rename('cb1')
    print(root_mean_squared_error(df_train.loc[df_train.index.isin(prd.index), target].sort_index(), prd))
    s_cb1.append(prd)
s_cb1 = pd.concat(s_cb1).sort_index()

38.69925444882886
38.7001656611936
38.72494393663837
38.692983930300905
38.7080609748558


# XGB1

In [10]:
results_0 = sc.cache_result(
    'cv_xgb1_0',
    lambda : eval_model({
        'model_params' : {'n_estimators': 1500, 'max_depth': 5, 'learning_rate': 0.03},
        'X_num': X_num, 'X_cat': X_cat, 'X_tgt': ['wc_i2'], 'tgt': {'cv': 24, 'smooth': 20, 'target_type': 'continuous', 'random_state': 123},
        #'validation_fraction': 0.1
    }, xgb_adapter, case = 0, sp = kf, device = 'cuda'), rerun = 0
)
results_0[0][1]

38.58607

In [11]:
results_1 = sc.cache_result(
    'cv_xgb1_1',
    lambda :eval_model({
        'model_params' : {'n_estimators': 2000, 'max_depth': 4, 'learning_rate': 0.03},
        'X_num': X_num, 'X_cat': X_cat,
        #'validation_fraction': 0.1
    }, xgb_adapter, case = 1, sp = kf, device = 'cuda'), rerun = 0
)
results_1[0][1]

39.233482

In [12]:
s_xgb1 = list()
for a, b in zip(results_0, results_1):
    prd = pd.concat([a[2], b[2]]).sort_index().rename('xgb1')
    print(root_mean_squared_error(df_train.loc[df_train.index.isin(prd.index), target].sort_index(), prd))
    s_xgb1.append(prd)
s_xgb1 = pd.concat(s_xgb1).sort_index()

38.702263
38.70285
38.730896
38.694828
38.715015


# LGB1

In [13]:
results_0 = sc.cache_result(
    'cv_lgb1_0',
    lambda : eval_model({
        'model_params' : {'n_estimators': 1000, 'learning_rate': 0.03},
        'X_num': X_num, 'X_cat': X_cat, 'X_tgt': ['wc_i2'], 'tgt': {'cv': 24, 'smooth': 20, 'target_type': 'continuous', 'random_state': 123},
        #'validation_fraction': 0.1
    }, lgb_adapter, case = 0, sp = kf), rerun = 0
)
results_0[0][1]

38.581233616920606

In [14]:
results_1 = sc.cache_result(
    'cv_lgb1_1',
    lambda : eval_model({
        'model_params' : {'n_estimators': 1000, 'learning_rate': 0.03},
        'X_num': X_num, 'X_cat': X_cat,
        #'validation_fraction': 0.1
    }, lgb_adapter, case = 1, sp = kf), rerun = 0
)
results_1[0][1]

39.21659948662713

In [15]:
s_lgb1 = list()
for a, b in zip(results_0, results_1):
    prd = pd.concat([a[2], b[2]]).sort_index().rename('lgb1')
    print(root_mean_squared_error(df_train.loc[df_train.index.isin(prd.index), target].sort_index(), prd))
    s_lgb1.append(prd)
s_lgb1 = pd.concat(s_lgb1).sort_index()

38.69524565122885
38.6935063504952
38.721391865182014
38.68683199170401
38.704066018691925


# Ensemble

In [16]:
df_cv = pd.concat([
    s_cb1, s_xgb1, s_lgb1
], axis=1)
df_cv.head()

,cb1,xgb1,lgb1
id,,,
0,82.029231,81.506348,81.654079
1,86.546309,86.527985,86.277539
2,81.949834,81.494560,82.112994
3,90.323875,88.683304,89.693178
4,82.410306,82.423264,82.103973


In [17]:
df_cv.apply(
    lambda x: root_mean_squared_error(df_train[target], x)
)

cb1     38.705083
xgb1    38.709167
lgb1    38.700210
dtype: float64

In [18]:
root_mean_squared_error(
    df_train[target], df_cv.mean(axis=1)
)

38.7002336272789

In [19]:
root_mean_squared_error(
    df_train[target], df_cv.dot([0.3, 0.1, 0.6])
)

38.69899891724877

In [20]:
def train_models(hparams0, hparams1, adapter, **args):
    return (
        sgml.train(df_train, hparams0, config, adapter, **args),
        sgml.train(df_train, hparams1, config, adapter, **args),
    )

In [22]:
reg_cb1 = sc.cache_result(
    'reg_cb1',
    lambda : train_models({
            'model_params' : {'n_estimators': 700, 'max_depth': 7, 'learning_rate': 0.1},
            'X_num': X_num, 'X_cat': X_cat, 'X_tgt': ['wc_i2'], 'tgt': {'cv': 24, 'smooth': 20, 'target_type': 'continuous', 'random_state': 123}
        }, {
            'model_params' : {'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.1},
            'X_num': X_num, 'X_cat': X_cat,
        }, cb_adapter, task_type = 'GPU'
    )
)

In [23]:
reg_xgb1 = sc.cache_result(
    'reg_xgb1',
    lambda : train_models({
            'model_params' : {'n_estimators': 1500, 'max_depth': 5, 'learning_rate': 0.03},
            'X_num': X_num, 'X_cat': X_cat, 'X_tgt': ['wc_i2'], 'tgt': {'cv': 24, 'smooth': 20, 'target_type': 'continuous', 'random_state': 123},
        }, {
            'model_params' : {'n_estimators': 2000, 'max_depth': 4, 'learning_rate': 0.03},
            'X_num': X_num, 'X_cat': X_cat,
        }, xgb_adapter, device = 'cuda'
    )
)

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

Round:   0%|          | 0/2000 [00:00<?, ?it/s]

In [24]:
reg_lgb1 = sc.cache_result(
    'reg_lgb1',
    lambda : train_models({
            'model_params' : {'n_estimators': 1000, 'learning_rate': 0.03},
            'X_num': X_num, 'X_cat': X_cat, 'X_tgt': ['wc_i2'], 'tgt': {'cv': 24, 'smooth': 20, 'target_type': 'continuous', 'random_state': 123},
        }, {
            'model_params' : {'n_estimators': 1000, 'learning_rate': 0.03},
            'X_num': X_num, 'X_cat': X_cat,\
        }, lgb_adapter
    )
)

Round:   0%|          | 0/1000 [00:00<?, ?it/s]

Round:   0%|          | 0/1000 [00:00<?, ?it/s]